# Hello Image Classification

A very basic introduction to OpenVINO that shows how to perform inference with an image classification model.

We use a pre-trained [MobileNetV3 model](https://docs.openvinotoolkit.org/latest/omz_models_model_mobilenet_v3_small_1_0_224_tf.html) from the [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo/). See the [TensorFlow to OpenVINO](101-tensorflow-to-openvino) tutorial to learn more about how this OpenVINO IR model was created.

## Imports

In [ ]:
import json

import cv2
import matplotlib.pyplot as plt
import numpy as np
from openvino.inference_engine import IECore

## Load the Model

In [ ]:
ie = IECore()
net = ie.read_network(
    model="model/v3-small_224_1.0_float.xml", weights="model/v3-small_224_1.0_float.bin"
)
exec_net = ie.load_network(network=net, device_name="CPU")

input_key = next(iter(exec_net.input_info))
output_key = next(iter(exec_net.outputs.keys()))

## Load an Image

In [ ]:
# The MobileNet network expects images in RGB format
image = cv2.cvtColor(cv2.imread(filename="data/coco.jpg"), code=cv2.COLOR_BGR2RGB)

# resize to MobileNet image shape
input_image = cv2.resize(src=image, dsize=(224, 224))

# reshape to network input shape
input_image = np.expand_dims(input_image.transpose(2, 0, 1), 0)
plt.imshow(image);

## Do Inference

In [ ]:
result = exec_net.infer(inputs={input_key: input_image})[output_key]
result_index = np.argmax(result)

In [ ]:
# Convert the inference result to a class name.
imagenet_classes = json.loads(open("utils/imagenet_class_index.json").read())

# The model description states that for this model, class 0 is background,
# so we add 1 to the network output to get the class name
imagenet_classes = {int(key) + 1: value for key, value in imagenet_classes.items()}
imagenet_classes[result_index]